In [1]:
import pyodbc
import pandas as pd
from pymongo import MongoClient
from flask import Flask , request ,jsonify
import threading
import time

In [2]:
app=Flask(__name__)

In [3]:
sql_conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-JGV3GCV\\SQLEXPRESS;"
    "DATABASE=STUDENTGUIDE;"
    "Trusted_Connection=yes;"
)

In [4]:
mongo_db = MongoClient("mongodb://localhost:27017/")["STUDENTGUIDE"]

In [5]:
@app.route('/EZZ312003', methods=["POST"])
def collection_data():
    table_name = "Collection"
    df = pd.read_sql(f"SELECT * FROM {table_name}", sql_conn)
    if not df.empty:
            # تنظيف وتحويل الأنواع
            df = df.applymap(lambda x:
                float(x) if isinstance(x, (int, float))
                else str(x) if pd.notna(x)
                else None
            )

            
            mongo_db[table_name].insert_many(df.to_dict("records"))
            print(f"✅ تم إدخال {len(df)} سجل في Mongo Collection اسمه {table_name}")
    else:
             print(f"⚠️ الجدول {table_name} فاضي في SQL")

In [6]:
def run_flask():
    app.run(port=5000, debug=False, use_reloader=False) 

In [ ]:
threading.Thread(target=run_flask).start() #علشان لو عندي كذا API يحجز مكان ف الذاكره 
time.sleep(1)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


C:\Users\C.M™\AppData\Local\Temp\ipykernel_16240\3244231686.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {table_name}", sql_conn)
C:\Users\C.M™\AppData\Local\Temp\ipykernel_16240\3244231686.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x:
[2025-09-06 19:46:33,021] ERROR in app: Exception on /EZZ312003 [POST]
Traceback (most recent call last):
  File "C:\Users\C.M™\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\C.M™\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packa

✅ تم إدخال 100214 سجل في Mongo Collection اسمه Collection


In [8]:
import requests
url= "http://127.0.0.1:5000/EZZ312003"
res = requests.post(url)
